# Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
import scipy.stats
import matplotlib
import geopandas as gpd
from datetime import datetime, timedelta
import jenkspy
import warnings
import random
warnings.filterwarnings("ignore")

# Define country and parameters

In [ ]:
# Select target country
country = 'Colombia'

# Set country-specific parameters: ISO codes and buffer size (in meters)
if country == 'Argentina':
    country_short = 'ARG'   # ISO 3-letter code
    country_code = 'AR'     # ISO 2-letter code
elif country == 'Chile':
    country_short = 'CHL'
    country_code = 'CL'
elif country == 'Colombia':
    country_short = 'COL'
    country_code = 'CO'
# Uncomment the following if Mexico is to be included in the analysis
# elif country == 'Mexico':
#     country_short = 'MEX'
#     country_code = 'MX'

# Set working directory

In [ ]:
# Define working directory path
wd = (
    '/your/path/to/working/directory/'
)

# Define some functions to generate time series of inflows, outflows

In [ ]:
def compute_flows(df_mov_evo, flow_type):
    """
    Compute either outflows or inflows aggregated by origin or destination.

    Parameters:
    - df_mov_evo: DataFrame containing movement data with columns 'O', 'D', and time series data.
    - flow_type: str, either 'outflows' (sum by origin) or 'inflows' (sum by destination).

    Returns:
    - df_flows: DataFrame aggregated by 'O' or 'D'.
    """

    # Initialise df_flows with unique IDs depending on flow_type
    if flow_type == 'outflows':
        df_flows = pd.DataFrame({'O': np.unique(df_mov_evo['O'])})
    elif flow_type == 'inflows':
        df_flows = pd.DataFrame({'D': np.unique(df_mov_evo['D'])})
    else:
        raise ValueError("flow_type must be 'outflows' or 'inflows'")

    # Prepare empty columns for the time series data initialised with NaNs
    time_columns = df_mov_evo.columns[2:]
    df_flows_add = pd.DataFrame({col: [np.nan] * len(df_flows) for col in time_columns})

    # Combine ID column with empty data columns
    df_flows = pd.concat([df_flows, df_flows_add], axis=1)

    # Iterate over each unique ID to compute aggregated flows
    for i in range(len(df_flows)):
        if flow_type == 'outflows':
            ID = df_flows.loc[i, 'O']
            df_subset = df_mov_evo[df_mov_evo['O'] == ID]
        else:  # inflows
            ID = df_flows.loc[i, 'D']
            df_subset = df_mov_evo[df_mov_evo['D'] == ID]

        # Sum values per column, ignoring NaNs
        for column in time_columns:
            values = df_subset[column].dropna()
            if values.empty:
                df_flows.loc[i, column] = np.nan
            else:
                df_flows.loc[i, column] = values.sum()

    return df_flows

In [ ]:
def compute_df_ts(df_flows, df_flows_baseline, initial_col):
    """
    Compute time series of movement sums and baseline sums from flow DataFrames,
    handle missing and zero values by interpolation using nearest observations,
    and calculate rolling averages and percentage changes.

    Parameters:
    - df_flows: DataFrame with flow data over time columns starting at initial_col
    - df_flows_baseline: Baseline DataFrame with same structure as df_flows
    - initial_col: int, index of the first time column in the DataFrames

    Returns:
    - df_ts: DataFrame with dates, movements, baseline, filled values, rolling means,
             and percentage change metrics.
    """

    evo_movs = []
    evo_movs_baseline = []

    # Aggregate sum of movements and baseline for each time column, ignoring NaNs
    for column in df_flows.columns[initial_col:]:
        sums_mov = []
        sums_baseline = []
        for i in range(len(df_flows)):
            val_mov = df_flows.loc[i, column]
            val_base = df_flows_baseline.loc[i, column]
            if not pd.isna(val_mov) and not pd.isna(val_base):
                sums_mov.append(val_mov)
                sums_baseline.append(val_base)
        if sums_mov:
            evo_movs.append(np.sum(sums_mov))
            evo_movs_baseline.append(np.sum(sums_baseline))
        else:
            evo_movs.append(np.nan)
            evo_movs_baseline.append(np.nan)

    # Create DataFrame with date and aggregated sums
    df_ts = pd.DataFrame({
        'date': df_flows.columns[initial_col:],
        'movements': evo_movs,
        'baseline': evo_movs_baseline
    })

    # Function to fill zeros and NaNs using mean of closest 15 observations in time series
    def fill_zeros_and_nans(series, fill_column_name):
        series.replace(0, np.nan, inplace=True)
        series.replace([np.inf, -np.inf], np.nan, inplace=True)

        is_na = series.isna()
        filled_series = series.copy()

        # Extract rows where values are not NaN for reference
        valid_idx = series.dropna().index
        valid_vals = series.dropna()

        # For each NaN, find 15 nearest valid observations by index distance and take their mean
        for idx in series[is_na].index:
            distances = abs(valid_idx - idx)
            nearest_idx = distances.nsmallest(15).index
            filled_series.loc[idx] = valid_vals.loc[nearest_idx].mean()

        return filled_series

    # Fill movements and baseline columns
    df_ts['movements_fill'] = fill_zeros_and_nans(df_ts['movements'], 'movements_fill')
    df_ts['rolling'] = df_ts['movements_fill'].rolling(window=15, min_periods=1).mean()

    df_ts['baseline_fill'] = fill_zeros_and_nans(df_ts['baseline'], 'baseline_fill')
    df_ts['rolling_baseline'] = df_ts['baseline_fill'].rolling(window=15, min_periods=1).mean()

    # Calculate percentage change between movements and baseline
    df_ts['perchange'] = (df_ts['movements_fill'] - df_ts['baseline_fill']) / df_ts['baseline_fill'] * 100
    df_ts['rolling_perchange'] = df_ts['perchange'].rolling(window=30, min_periods=1).mean()

    return df_ts

In [ ]:
def compute_df_ts_month(df_ts):
    """
    Aggregates daily time series data into monthly summaries (assuming each month = 30 days).

    Parameters:
    -----------
    df_ts : pd.DataFrame
        Input DataFrame containing daily time series data with the following columns:
        - 'date'
        - 'movements'
        - 'baseline'
        - 'movements_fill'
        - 'baseline_fill'

    Returns:
    --------
    df_ts_month : pd.DataFrame
        Monthly aggregated DataFrame with columns:
        - 'month_no': Sequential month index
        - 'month_start': Date of the first day of each month
        - 'movements': Sum of 'movements' in the month
        - 'baseline': Sum of 'baseline' in the month
        - 'movements_fill': Sum of 'movements_fill' in the month
        - 'baseline_fill': Sum of 'baseline_fill' in the month
        - 'perchange': Percent change between movements_fill and baseline_fill
    """
    
    # Number of months assuming 30-day months
    num_months = int(len(df_ts) / 30)

    # Initialize DataFrame with month indices
    df_ts_month = pd.DataFrame({'month_no': range(num_months)})

    # Extract the first date of each month
    df_ts_month['month_start'] = [
        df_ts.loc[i * 30, 'date'] for i in range(num_months)
    ]

    # Sum each variable over each 30-day month
    df_ts_month['movements'] = [
        np.sum(df_ts.loc[i * 30:(i + 1) * 30 - 1, 'movements']) for i in range(num_months)
    ]
    df_ts_month['baseline'] = [
        np.sum(df_ts.loc[i * 30:(i + 1) * 30 - 1, 'baseline']) for i in range(num_months)
    ]
    df_ts_month['movements_fill'] = [
        np.sum(df_ts.loc[i * 30:(i + 1) * 30 - 1, 'movements_fill']) for i in range(num_months)
    ]
    df_ts_month['baseline_fill'] = [
        np.sum(df_ts.loc[i * 30:(i + 1) * 30 - 1, 'baseline_fill']) for i in range(num_months)
    ]

    # Calculate percentage change from baseline_fill to movements_fill
    df_ts_month['perchange'] = [
        (df_ts_month.loc[i, 'movements_fill'] - df_ts_month.loc[i, 'baseline_fill']) / 
        df_ts_month.loc[i, 'baseline_fill'] * 100
        for i in range(num_months)
    ]

    return df_ts_month

# Read some additional data

In [ ]:
# Load baseline population data with exogenous variables from a GeoPackage file
baseline_pop_imput = gpd.read_file(
    wd + '/data/outputs/' + country_short + '/grids-with-data/movcell-baseline-imput-pop-with-exo-var/movcell-baseline-imput-pop-with-exo-var.gpkg'
)

In [ ]:
# Set configuration flags
dist = True
raw = False
adjust = True

# Determine string suffix for 'dist'
if dist:
    dist = '_dist'
else:
    dist = ''

# Determine string suffix for 'raw' and conditionally for 'adjust'
if raw:
    raw = '_raw'
    adjust = ''  # 'adjust' is ignored if 'raw' is True
else:
    raw = ''
    if adjust:
        adjust = '_adjust'
    else:
        adjust = ''

# Read movement data as time series for each tile

In [ ]:
# Read mobility evolution data from CSV file, drop the first unnamed index column
df_mov_evo = pd.read_csv(
    wd + '/data/outputs/' + country_short + '/evo/mov_evo' + dist + raw + adjust + '.csv'
).drop('Unnamed: 0', axis=1)

# Read baseline mobility evolution data, drop the first unnamed index column
df_mov_evo_baseline = pd.read_csv(
    wd + '/data/outputs/' + country_short + '/evo/mov_evo_baseline' + dist + raw + '.csv'
).drop('Unnamed: 0', axis=1)

In [ ]:
 # Set the type of flow to analyze ('movs', 'inflows', or 'outflows')
flows = 'outflows'  # change to 'movs', 'inflows', or 'outflows' as needed

# Load appropriate flow data depending on the selected flow type
if flows in ['inflows', 'outflows']:
    # Load computed flow data and baseline from CSV, dropping the index column
    df_flows = pd.read_csv(
        wd + '/data/outputs/' + country_short + '/mov-analysis/' +
        flows + dist + raw + adjust + '.csv'
    ).drop('Unnamed: 0', axis=1)

    df_flows_baseline = pd.read_csv(
        wd + '/data/outputs/' + country_short + '/mov-analysis/' +
        flows + '_baseline' + dist + raw + '.csv'
    ).drop('Unnamed: 0', axis=1)

    # Set the initial column index for further processing
    initial_col = 1

else:
    # Use raw mobility evolution data directly if 'movs' is selected
    df_flows = df_mov_evo
    df_flows_baseline = df_mov_evo_baseline
    initial_col = 2

# Compute total inflows or outflows, as a sum, handling missing values

And visualise

In [ ]:
# Compute the time series dataframe using flow data
df_ts = compute_df_ts(df_flows, df_flows_baseline, initial_col)

# Create a new figure and axis for plotting
fig, ax = plt.subplots()

# Customise axis ticks appearance
ax.tick_params(axis='both', which='both', width=0, length=0, color='k', labelsize=20, pad=9)

# Compute monthly aggregates from daily time series
df_ts_month = compute_df_ts_month(df_ts)

# Apply a rolling mean (window=1 has no smoothing effect)
df_ts_month.loc[:, 'rolling_perchange'] = df_ts_month['perchange'].rolling(window=1).mean()

# Print debug information: length of data vs x-axis values
print(len(df_ts_month), len(np.arange(len(df_ts_month['rolling_perchange'])) * 30))

# Plot the rolling percent change
ax.plot(np.arange(len(df_ts_month['rolling_perchange'])) * 30,
        df_ts_month['rolling_perchange'], color='black', lw=2, zorder=3)

# Plot a horizontal reference line at 0%
ax.plot(np.arange(len(df_ts_month['rolling_perchange']) * 30),
        np.zeros(len(df_ts_month['rolling_perchange']) * 30),
        linestyle=':', color='k')

# Determine y-axis bounds
ymin = int(min([-100, np.min(df_ts_month['rolling_perchange'])]))
ymax = int(max([101, np.max(df_ts_month['rolling_perchange']) + 1]))

# Define x-tick positions and labels (approx. every 6 months)
xticks = []
xticks_labels = ['Apr 2020', 'Oct 2020', 'Apr 2021', 'Oct 2021', 'April 2022']
for i in range(0, len(df_ts['rolling_perchange'])):
    if i % 183 == 0:
        xticks.append(i)
ax.set_xticks(xticks, xticks_labels)
ax.tick_params(axis='x', bottom=True, labelsize=10, pad=6, rotation=90)

# Define y-tick positions (every 25%)
yticks = []
for i in range(ymin, ymax):
    if i % 25 == 0:
        yticks.append(i)
ax.set_yticks(yticks, yticks)

# Draw horizontal gridlines at each y-tick
for y in yticks:
    ax.plot([0, len(df_ts['rolling_perchange'])], [y, y], color='gray', lw=0.7, zorder=0)

# Customise y-axis tick appearance
ax.tick_params(axis='y', labelsize=10, pad=6, rotation=0)

# Optional: Save figure to file
# plt.savefig(wd + '/plots/evolution/' + flows + '/total/' + country_short + '/evo' + dist + raw + adjust + '.pdf', bbox_inches='tight')

# Display the plot
plt.show()

# By density class

In [ ]:
# Number of classes to divide the density into
n_class_density = 5

# Calculate Jenks natural breaks for the 'density' column (excluding NaNs)
breaks_density = jenkspy.jenks_breaks(
    baseline_pop_imput.dropna(subset=['density'])['density'], 
    n_classes=n_class_density
)

# Adjust the first break slightly to include minimum values properly
breaks_density[0] = breaks_density[0] - 10**(-10)

# Assign density classes using quantile-based discretization (commented out here)
# baseline_pop_imput['class_density'] = pd.qcut(
#     baseline_pop_imput['density'], 
#     q=n_class_density, 
#     labels=[i for i in range(n_class_density)]
# )

# Assign density classes using Jenks natural breaks
baseline_pop_imput['class_density'] = pd.cut(
    baseline_pop_imput['density'], 
    bins=breaks_density, 
    labels=[i for i in range(n_class_density)]
)

# Convert categorical labels to numeric
baseline_pop_imput['class_density'] = pd.to_numeric(baseline_pop_imput['class_density'])

# Get unique density classes ignoring NaN values
class_density = np.unique(baseline_pop_imput['class_density'])

# Update the number of density classes based on unique non-NaN values
n_class_density = len(class_density[~np.isnan(class_density)])


# Prepare data for boxplots by density

In [ ]:
# Initialise an array to store monthly time series data by density class
df_ts_month_class_density = np.zeros((n_class_density, len(df_ts_month)))

# Lists to hold percentage changes for first and last months by density class
first_month_perchange_density = []
last_month_perchange_density = []

for i in range(n_class_density):
    # Get indices of locations belonging to the current density class
    indexes = set(baseline_pop_imput[baseline_pop_imput['class_density'] == i].index)
    
    # Create a mask to filter flows data depending on the flow type
    if flows == 'movs':
        mask = df_flows['O'].isin(indexes) | df_flows['D'].isin(indexes)
    elif flows == 'outflows':
        mask = df_flows['O'].isin(indexes)
    elif flows == 'inflows':
        mask = df_flows['D'].isin(indexes)
        
    # Filter flows and baseline data for the current density class
    df_flows_class_density = df_flows[mask].reset_index(drop=True)
    df_flows_class_density_baseline = df_flows_baseline[mask].reset_index(drop=True)
    
    # Extract columns corresponding to June 2020 (first month of interest)
    columns_first_month = [col for col in df_flows_class_density.columns if '2020-06' in col]
    first_month = df_flows_class_density.loc[0:len(df_flows_class_density), columns_first_month].to_numpy().flatten()

    columns_first_month_baseline = [col for col in df_flows_class_density_baseline.columns if '2020-06' in col]
    first_month_baseline = df_flows_class_density_baseline.loc[0:len(df_flows_class_density_baseline), columns_first_month_baseline].to_numpy().flatten()

    # Calculate percentage change for first month, ignoring NaNs
    first_month_perchange = []
    for j in range(len(first_month)):
        x = (first_month[j] - first_month_baseline[j]) / first_month_baseline[j] * 100
        if not pd.isna(x):
            first_month_perchange.append(x)
    
    # If it's the first density class and the sample size is large, downsample to 30 randomly
    if i == 0 and len(first_month_perchange) > 1000:
        first_month_perchange = np.random.choice(first_month_perchange, 30, replace=False)
    
    first_month_perchange_density.append(np.array(first_month_perchange))

    # Extract columns corresponding to March 2022 (last month of interest)
    columns_last_month = [col for col in df_flows_class_density.columns if '2022-03' in col]
    last_month = df_flows_class_density.loc[0:len(df_flows_class_density), columns_last_month].to_numpy().flatten()

    columns_last_month_baseline = [col for col in df_flows_class_density_baseline.columns if '2022-03' in col]
    last_month_baseline = df_flows_class_density_baseline.loc[0:len(df_flows_class_density_baseline), columns_last_month_baseline].to_numpy().flatten()

    # Calculate percentage change for last month, ignoring NaNs
    last_month_perchange = []
    for j in range(len(last_month)):
        x = (last_month[j] - last_month_baseline[j]) / last_month_baseline[j] * 100
        if not pd.isna(x):
            last_month_perchange.append(x)

    # This line appears to be duplicated and intended for the first month, but it is placed here again
    if i == 0 and len(first_month_perchange) > 1000:
        first_month_perchange = np.random.choice(first_month_perchange, 30, replace=False)
    
    last_month_perchange_density.append(np.array(last_month_perchange))

# Convert lists to numpy arrays for further processing
first_month_perchange_density = np.array(first_month_perchange_density)
last_month_perchange_density = np.array(last_month_perchange_density)


In [ ]:
# Create a figure and axis with specified size and tight layout
fig, ax = plt.subplots(figsize=(12, 6), layout='tight')

# Plot a horizontal baseline at y=0 for reference
ax.plot([-0.5, 14.5], [0, 0], color='k', linewidth=2, zorder=0)

# Generate colors for each density class using the viridis colormap
colors = mcp.gen_color(cmap='viridis', n=n_class_density)

# Prepare data for first month percentage changes
data = first_month_perchange_density
data_boxplot = []
palette = mcp.gen_color(cmap='viridis', n=n_class_density)
means, medians, modes = [], [], []

for i in range(n_class_density):
    if i == 0:
        sample = data[i]
        try:
            # Calculate 7th and 93rd percentiles to filter outliers
            Q1 = np.percentile(sample, 7)
            Q3 = np.percentile(sample, 93)
            IQR = Q3 - Q1
            filter_sample = sample[(sample >= Q1) & (sample <= Q3)]
            print(len(filter_sample))
            # Resample with replacement to get a fixed sample size of 300
            sample = np.random.choice(filter_sample, 300, replace=True)
        except:
            print('no')
    else:
        sample = list(data[i])
    
    data_boxplot.append(sample)
    
    # Calculate stats for plotting or reporting
    means.append(np.mean(sample))
    medians.append(np.median(sample))
    modes.append(scipy.stats.mode(sample)[0][0])
    
    # Prepare deviation quantiles (not currently used for plotting)
    df_data = pd.DataFrame({
        'data': sample,
        'dev': [np.abs(sample[j] - np.median(sample)) for j in range(len(sample))]
    })
    df_data['quantile'] = pd.qcut(df_data['dev'], n_class_density, labels=False, duplicates='drop')
    max_dev = max([abs(x - np.median(sample)) for x in sample])

# Create boxplots for first month data with some styling and positioning
bplot = ax.boxplot(
    data_boxplot,
    positions=[(i * 3 + 1) - 0.65 for i in range(n_class_density)],
    notch=True,
    whis=[5, 95],
    showfliers=False,
    widths=1,
    boxprops=dict(linewidth=2),
    flierprops=dict(markeredgecolor='gray', marker='o', markersize=3),
    whiskerprops=dict(linewidth=2),
    medianprops=dict(linewidth=4),
    zorder=6,
    patch_artist=True
)

# Set boxplot face colors and transparency
for patch, color in zip(bplot['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.9)
    patch.set_zorder(5)

# Now prepare data for last month percentage changes
data = last_month_perchange_density
colors = mcp.gen_color(cmap='viridis', n=n_class_density)

data_boxplot = []
palette = mcp.gen_color(cmap='viridis', n=n_class_density)
means, medians, modes = [], [], []

for i in range(n_class_density):
    if i == 0:
        sample = data[i]
        # Filter outliers between 7th and 93rd percentiles and resample 500 points
        Q1 = np.percentile(sample, 7)
        Q3 = np.percentile(sample, 93)
        IQR = Q3 - Q1
        filter_sample = sample[(sample >= Q1) & (sample <= Q3)]
        print(len(filter_sample))
        sample = np.random.choice(filter_sample, 500, replace=True)
    else:
        sample = list(data[i])
    
    data_boxplot.append(sample)
    
    means.append(np.mean(sample))
    medians.append(np.median(sample))
    modes.append(scipy.stats.mode(sample)[0][0])
    
    df_data = pd.DataFrame({
        'data': sample,
        'dev': [np.abs(sample[j] - np.median(sample)) for j in range(len(sample))]
    })
    df_data['quantile'] = pd.qcut(df_data['dev'], n_class_density, labels=False, duplicates='drop')
    max_dev = max([abs(x - np.median(sample)) for x in sample])

# Draw patterned boxplots (hatched) with partial transparency for last month data
bplot = ax.boxplot(
    data_boxplot,
    positions=[(i * 3 + 1) + 0.65 for i in range(n_class_density)],
    notch=True,
    whis=[5, 95],
    showfliers=False,
    widths=1,
    boxprops=dict(linewidth=0),
    flierprops=dict(markeredgecolor='gray', marker='o', markersize=3),
    whiskerprops=dict(linewidth=0),
    medianprops=dict(linewidth=0),
    zorder=5,
    patch_artist=True
)
for patch, color in zip(bplot['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.4)
    patch.set(hatch='///')
    patch._hatch_color = matplotlib.colors.to_rgba('k')
    patch.set_zorder(6)

# Overlay boxplots for last month data with full outlines and median lines
bplot = ax.boxplot(
    data_boxplot,
    positions=[(i * 3 + 1) + 0.65 for i in range(n_class_density)],
    notch=True,
    whis=[5, 95],
    showfliers=False,
    widths=1,
    boxprops=dict(linewidth=2),
    flierprops=dict(markeredgecolor='gray', marker='o', markersize=3),
    whiskerprops=dict(linewidth=2),
    medianprops=dict(linewidth=4),
    zorder=7,
    patch_artist=False
)

# Set axis limits
ax.set_xlim([-1, 15])
ax.set_ylim([-100, 200])

# Configure tick parameters for axes
ax.tick_params(axis='x', which='both', labelsize=32, pad=12)
ax.tick_params(axis='y', which='both', labelsize=32, pad=12)
ax.tick_params(axis='both', which='both', width=0, length=0, color='k')

# Set custom x-ticks and their labels
ax.set_xticks([1, 4, 7, 10, 13], [1, 2, 3, 4, 5])

# Save the figure (commented out)
# plt.savefig(wd + '/plots/boxplots/' + country_short + '_density.pdf', bbox_inches='tight')

plt.show()

# By rdi class

In [ ]:
n_class_rdi = 3  # Number of classes for RDI (Relative Deprivation Index)

# Compute Jenks natural breaks for 'rdi' values, ignoring NaNs
breaks_rdi = jenkspy.jenks_breaks(baseline_pop_imput.dropna(subset=['rdi'])['rdi'], n_classes=n_class_rdi)

# Adjust the lowest break slightly downwards to include boundary values properly
breaks_rdi[0] = breaks_rdi[0] - 10**(-10)

# Assign RDI classes based on Jenks breaks, labeling classes 0 to n_class_rdi-1
# (commented out alternative quantile cut below)
# baseline_pop_imput['class_rdi'] = pd.qcut(baseline_pop_imput['rdi'], q=n_class_rdi, labels=[i for i in range(n_class_rdi)])
baseline_pop_imput['class_rdi'] = pd.cut(baseline_pop_imput['rdi'], bins=breaks_rdi, labels=[i for i in range(n_class_rdi)])

# Convert class labels from categorical to numeric type
baseline_pop_imput['class_rdi'] = pd.to_numeric(baseline_pop_imput['class_rdi'])

# Identify unique RDI classes and count non-NaN classes
class_rdi = np.unique(baseline_pop_imput['class_rdi'])
n_class_rdi = len(class_rdi[~np.isnan(class_rdi)])


# Prepare data for boxplots by RDI

In [ ]:
# Initialise array to hold time series data for each RDI class
df_ts_month_class_rdi = np.zeros((n_class_rdi, len(df_ts_month)))

# Lists to store percentage changes for first and last months per RDI class
first_month_perchange_rdi = []
last_month_perchange_rdi = []

for i in range(n_class_rdi):

    # Get indexes of regions belonging to the current RDI class
    indexes = set(baseline_pop_imput[baseline_pop_imput['class_density'] == i].index)

    # Create mask to filter flows based on the type (movements, outflows, inflows)
    if flows == 'movs':
        mask = df_flows['O'].isin(indexes) | df_flows['D'].isin(indexes)
    elif flows == 'outflows':
        mask = df_flows['O'].isin(indexes)
    elif flows == 'inflows':
        mask = df_flows['D'].isin(indexes)

    # Subset flows and baseline flows data for the current RDI class
    df_flows_class_rdi = df_flows[mask].reset_index(drop=True)
    df_flows_class_rdi_baseline = df_flows_baseline[mask].reset_index(drop=True)
    
    # Extract columns corresponding to May 2020 (first month of interest)
    columns_first_month = [col for col in df_flows_class_rdi.columns if '2020-05' in col]
    first_month = df_flows_class_rdi.loc[0:len(df_flows_class_rdi), columns_first_month].to_numpy().flatten()

    columns_first_month_baseline = [col for col in df_flows_class_rdi_baseline.columns if '2020-05' in col]
    first_month_baseline = df_flows_class_rdi_baseline.loc[0:len(df_flows_class_rdi_baseline), columns_first_month_baseline].to_numpy().flatten()

    # Calculate percentage change for first month compared to baseline
    first_month_perchange = []
    for j in range(len(first_month)):
        x = (first_month[j] - first_month_baseline[j]) / first_month_baseline[j] * 100
        if not pd.isna(x):
            first_month_perchange.append(x)
    # Downsample if too many samples in class 0
    if i == 0 and len(first_month_perchange) > 1000:
        first_month_perchange = np.random.choice(first_month_perchange, 1000, replace=False)
    first_month_perchange_rdi.append(np.array(first_month_perchange))

    # Extract columns corresponding to March 2022 (last month of interest)
    columns_last_month = [col for col in df_flows_class_rdi.columns if '2022-03' in col]
    last_month = df_flows_class_rdi.loc[0:len(df_flows_class_rdi), columns_last_month].to_numpy().flatten()

    columns_last_month_baseline = [col for col in df_flows_class_rdi_baseline.columns if '2022-03' in col]
    last_month_baseline = df_flows_class_rdi_baseline.loc[0:len(df_flows_class_rdi_baseline), columns_last_month_baseline].to_numpy().flatten()

    # Calculate percentage change for last month compared to baseline
    last_month_perchange = []
    for j in range(len(last_month)):
        x = (last_month[j] - last_month_baseline[j]) / last_month_baseline[j] * 100
        if not pd.isna(x):
            last_month_perchange.append(x)
    # Downsample if too many samples in class 0
    if i == 0 and len(last_month_perchange) > 1000:
        last_month_perchange = np.random.choice(last_month_perchange, 1000, replace=False)
    last_month_perchange_rdi.append(np.array(last_month_perchange))

# Convert lists to numpy arrays
first_month_perchange_rdi = np.array(first_month_perchange_rdi)
last_month_perchange_rdi = np.array(last_month_perchange_rdi)


In [ ]:
fig, ax = plt.subplots(figsize=(12, 6), layout='tight')

data = first_month_perchange_rdi
ax.plot([-0.5, 14.5], [0, 0], color='k', linewidth=2, zorder=0)
colors = mcp.gen_color(cmap='viridis', n=n_class_rdi)

data_boxplot = []
palette = mcp.gen_color(cmap='viridis', n=n_class_rdi)
means = []
medians = []
modes = []

for i in reversed(range(n_class_rdi)):
    if i == 0:
        sample = data[i]
        q1 = np.percentile(sample, 10)
        q3 = np.percentile(sample, 90)
        iqr = q3 - q1
        filter_sample = sample[(sample >= q1) & (sample <= q3)]
        sample = np.random.choice(filter_sample, 400, replace=False)
    else:
        sample = list(data[i])

    data_boxplot.append(sample)
    pos = (i * 5 + 2) - 1
    width = 1
    means.append(np.mean(sample))
    medians.append(np.median(sample))
    modes.append(scipy.stats.mode(sample)[0][0])

    df_data = pd.DataFrame({
        'data': sample,
        'dev': [np.abs(sample[j] - np.median(sample)) for j in range(len(sample))]
    })
    df_data['quantile'] = pd.qcut(
        df_data['dev'], n_class_rdi, labels=False, duplicates='drop'
    )
    max_dev = max([abs(x - np.median(sample)) for x in sample])

bplot = ax.boxplot(
    data_boxplot,
    positions=[((i * 2) * 3 + 1) - 0.65 for i in range(n_class_rdi)],
    notch=True,
    whis=[5, 95],
    showfliers=False,
    widths=width,
    boxprops=dict(linewidth=2),
    flierprops=dict(markeredgecolor='gray', marker='o', markersize=3),
    whiskerprops=dict(linewidth=2),
    medianprops=dict(linewidth=4),
    zorder=6,
    patch_artist=True,
)
for patch, color in zip(bplot['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.9)

data = last_month_perchange_rdi
colors = mcp.gen_color(cmap='viridis', n=n_class_rdi)

data_boxplot = []
palette = mcp.gen_color(cmap='viridis', n=n_class_rdi)
means = []
medians = []
modes = []

for i in reversed(range(n_class_rdi)):
    if i == 0:
        sample = data[i]
        q1 = np.percentile(sample, 10)
        q3 = np.percentile(sample, 90)
        iqr = q3 - q1
        filter_sample = sample[(sample >= q1) & (sample <= q3)]
        sample = np.random.choice(filter_sample, 400, replace=False)
    else:
        sample = list(data[i])

    data_boxplot.append(sample)
    pos = i * 6 + 1 + 1
    width = 1
    means.append(np.mean(sample))
    medians.append(np.median(sample))
    modes.append(scipy.stats.mode(sample)[0][0])

    df_data = pd.DataFrame({
        'data': sample,
        'dev': [np.abs(sample[j] - np.median(sample)) for j in range(len(sample))]
    })
    df_data['quantile'] = pd.qcut(
        df_data['dev'], n_class_rdi, labels=False, duplicates='drop'
    )
    max_dev = max([abs(x - np.median(sample)) for x in sample])

bplot = ax.boxplot(
    data_boxplot,
    positions=[((i * 2) * 3 + 1) + 0.65 for i in range(n_class_rdi)],
    notch=True,
    whis=[5, 95],
    showfliers=False,
    widths=width,
    boxprops=dict(linewidth=0),
    flierprops=dict(markeredgecolor='gray', marker='o', markersize=3),
    whiskerprops=dict(linewidth=0),
    medianprops=dict(linewidth=0),
    zorder=6,
    patch_artist=True,
)
for patch, color in zip(bplot['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.4)
    patch.set(hatch='///')
    patch._hatch_color = matplotlib.colors.to_rgba('k')

bplot = ax.boxplot(
    data_boxplot,
    positions=[((i * 2) * 3 + 1) + 0.65 for i in range(n_class_rdi)],
    notch=True,
    whis=[5, 95],
    showfliers=False,
    widths=width,
    boxprops=dict(linewidth=2),
    flierprops=dict(markeredgecolor='gray', marker='o', markersize=3),
    whiskerprops=dict(linewidth=2),
    medianprops=dict(linewidth=4),
    zorder=7,
    patch_artist=False,
)

ax.set_ylim([-100, 200])

ax.tick_params(axis='x', which='both', labelsize=32, pad=12)
ax.tick_params(axis='y', which='both', labelsize=32, pad=12)
ax.tick_params(axis='both', which='both', width=0, length=0, color='k')

ax.set_xticks([1, 7, 13], [1, 2, 3])

plt.savefig(wd + '/plots/boxplots/' + country_short + '_rdi.pdf', bbox_inches='tight')

plt.show()